### Churn Analysis on Subscription Data

In [1]:
# Import Libs
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from datetime import datetime


In [2]:
# Load the dataset from a CSV file
file_path = 'student_data.csv'  # Path to the dataset
df = pd.read_csv(file_path)

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   student_id           199 non-null    int64 
 1   join_date            199 non-null    object
 2   membership_type      199 non-null    object
 3   total_classes        199 non-null    int64 
 4   avg_classes_per_mo   199 non-null    int64 
 5   last_class_date      199 non-null    object
 6   no_show_count        199 non-null    int64 
 7   monthly_payment      199 non-null    int64 
 8   outstanding_balance  199 non-null    object
dtypes: int64(5), object(4)
memory usage: 14.1+ KB


,student_id,join_date,membership_type,total_classes,avg_classes_per_mo,last_class_date,no_show_count,monthly_payment,outstanding_balance
0,674339,10/29/2023,Monthly,207,2,12/20/2024,13,97,Yes
1,525480,10/4/2022,Drop-in,170,10,8/26/2024,26,90,No
2,209650,4/19/2021,Monthly,71,2,10/28/2024,26,77,Yes
3,886849,1/31/2024,Annual,67,11,7/27/2024,4,101,Yes
4,762112,6/25/2024,Monthly,72,9,9/7/2024,21,77,Yes


In [4]:
df.isnull().sum()
df.isna().sum()

student_id             0
join_date              0
membership_type        0
total_classes          0
avg_classes_per_mo     0
last_class_date        0
no_show_count          0
monthly_payment        0
outstanding_balance    0
dtype: int64

In [5]:
# Feature Engineering
df["join_date"] = pd.to_datetime(df["join_date"])
df["last_class_date"] = pd.to_datetime(df["last_class_date"])
current_date = datetime(2024, 12, 31) # Example current date for the sake of this example
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   student_id           199 non-null    int64         
 1   join_date            199 non-null    datetime64[ns]
 2   membership_type      199 non-null    object        
 3   total_classes        199 non-null    int64         
 4   avg_classes_per_mo   199 non-null    int64         
 5   last_class_date      199 non-null    datetime64[ns]
 6   no_show_count        199 non-null    int64         
 7   monthly_payment      199 non-null    int64         
 8   outstanding_balance  199 non-null    object        
dtypes: datetime64[ns](2), int64(5), object(2)
memory usage: 14.1+ KB


In [7]:
# Calculate the number of days since joining
df["tenure_days"] = (current_date - df["join_date"]).dt.days
df["time_since_last_class"] = (current_date - df["last_class_date"]).dt.days
df.head()


,student_id,join_date,membership_type,total_classes,avg_classes_per_mo,last_class_date,no_show_count,monthly_payment,outstanding_balance,tenure_days,time_since_last_class
0,674339,2023-10-29,Monthly,207,2,2024-12-20,13,97,Yes,429,11
1,525480,2022-10-04,Drop-in,170,10,2024-08-26,26,90,No,819,127
2,209650,2021-04-19,Monthly,71,2,2024-10-28,26,77,Yes,1352,64
3,886849,2024-01-31,Annual,67,11,2024-07-27,4,101,Yes,335,157
4,762112,2024-06-25,Monthly,72,9,2024-09-07,21,77,Yes,189,115


In [10]:
# Preprocess the data
# This is sample data does not include a target variable "churn" adding for ddemonstration
# Encode categorical variables (Membership type)
le_member = LabelEncoder()
df["membership_type_encoder"] = le_member.fit_transform(df["membership_type"])

# Encode the outstanding balance
df["outstanding_balance_encoded"] = df["outstanding_balance"].map({"Yes": 1, "No": 0})

df.head()

,student_id,join_date,membership_type,total_classes,avg_classes_per_mo,last_class_date,no_show_count,monthly_payment,outstanding_balance,tenure_days,time_since_last_class,membership_type_encoder,outstanding_balance_encoded
0,674339,2023-10-29,Monthly,207,2,2024-12-20,13,97,Yes,429,11,2,1
1,525480,2022-10-04,Drop-in,170,10,2024-08-26,26,90,No,819,127,1,0
2,209650,2021-04-19,Monthly,71,2,2024-10-28,26,77,Yes,1352,64,2,1
3,886849,2024-01-31,Annual,67,11,2024-07-27,4,101,Yes,335,157,0,1
4,762112,2024-06-25,Monthly,72,9,2024-09-07,21,77,Yes,189,115,2,1
